In [ ]:
! pip install import_ipynb

  Created wheel for import-ipynb: filename=import_ipynb-0.1.3-py3-none-any.whl size=2975 sha256=3d1c85d3bb9a1b2a4d3e4fef94cfef36ae5ccbe704aaae19549fcf229936f10f
  Stored in directory: /root/.cache/pip/wheels/b1/5e/dc/79780689896a056199b0b9f24471e3ee184fbd816df355d5f0
Successfully built import-ipynb


In [ ]:
import import_ipynb
import numpy as np
import pandas as pd
from drive.MyDrive.python.Project.EPL_predict.EPL_prepare import *

importing Jupyter notebook from /content/drive/MyDrive/python/Project/EPL_predict/EPL_prepare.ipynb


In [ ]:
def data_load(season):
    df = pd.read_csv("/content/drive/MyDrive/python/Project/EPL_predict/data/season_data_{}_{}.csv".format(season, season+1), index_col='Unnamed: 0')
    return df

def data_save(df, season):
    df.to_csv("/content/drive/MyDrive/python/Project/EPL_predict/data/season_data_{}_{}.csv".format(season, season+1))

def fullseason(*arg):
    for num, season in enumerate(arg):
        if num == 0:
            df_full = data_load(season)
            df_full = lower_name(df_full)
            df_full = date_prepare(df_full)
            df_full = form_prepare(df_full)
            df_full = df_full.sort_values(by='date').reset_index(drop=True)
        else:
            df = data_load(season)
            df = lower_name(df)
            df = date_prepare(df)
            df = form_prepare(df)
            df = df.sort_values(by='date').reset_index(drop=True)
            df_full = pd.concat((df_full, df), axis=0)
    return df_full

def team_select(df_full, team):
    home_a = df_full.loc[df_full['home_team'] == team]
    away_a = pd.DataFrame()
    away_a['home_team'] = df_full.loc[df_full['away_team'] == team, 'away_team']
    away_a['away_team'] = df_full.loc[df_full['away_team'] == team, 'home_team']
    away_a['home_score'] = df_full.loc[df_full['away_team'] == team, 'away_score']
    away_a['away_score'] = df_full.loc[df_full['away_team'] == team, 'home_score']
    away_a['home_open_play'] = df_full.loc[df_full['away_team'] == team, 'away_open_play']
    away_a['away_open_play'] = df_full.loc[df_full['away_team'] == team, 'home_open_play']
    away_a['home_setpiece'] = df_full.loc[df_full['away_team'] == team, 'away_setpiece']
    away_a['away_setpiece'] = df_full.loc[df_full['away_team'] == team, 'home_setpiece']
    away_a['home_counterattack'] = df_full.loc[df_full['away_team'] == team, 'away_counterattack']
    away_a['away_counterattack'] = df_full.loc[df_full['away_team'] == team, 'home_counterattack']
    away_a['home_penalty'] = df_full.loc[df_full['away_team'] == team, 'away_penalty']
    away_a['away_penalty'] = df_full.loc[df_full['away_team'] == team, 'home_penalty']
    away_a['home_cross'] = df_full.loc[df_full['away_team'] == team, 'away_cross']
    away_a['away_cross'] = df_full.loc[df_full['away_team'] == team, 'home_cross']
    away_a['home_longball'] = df_full.loc[df_full['away_team'] == team, 'away_longball']
    away_a['away_longball'] = df_full.loc[df_full['away_team'] == team, 'home_longball']
    away_a['home_throughball'] = df_full.loc[df_full['away_team'] == team, 'away_throughball']
    away_a['away_throughball'] = df_full.loc[df_full['away_team'] == team, 'home_throughball']
    away_a['home_stortball'] = df_full.loc[df_full['away_team'] == team, 'away_stortball']
    away_a['away_stortball'] = df_full.loc[df_full['away_team'] == team, 'home_stortball']
    away_a['home_defense'] = df_full.loc[df_full['away_team'] == team, 'away_defense']
    away_a['away_defense'] = df_full.loc[df_full['away_team'] == team, 'home_defense']
    away_a['home_attack'] = df_full.loc[df_full['away_team'] == team, 'away_attack']
    away_a['away_attack'] = df_full.loc[df_full['away_team'] == team, 'home_attack']
    away_a['date'] = df_full.loc[df_full['away_team'] == team, 'date']
    df_team = pd.concat((home_a[away_a.columns], away_a), axis=0)
    df_team = df_team.sort_values(by='date').reset_index(drop=True)
    return df_team

def cup_team(df_cup, team):
    home = df_cup.loc[df_cup['home_team'] == team, ['home_team', 'date']]
    away = pd.DataFrame()
    away[['home_team', 'date']] = df_cup.loc[df_cup['away_team'] == team, ['away_team', 'date']]
    df = pd.concat((home,away), axis=0).sort_values(by='date').reset_index(drop=True)
    return df

def update_team(*arg):
    for season in arg:
        df = data_load(season)
        df = lower_name(df)
        df = date_prepare(df)
        df = form_prepare(df)

        home_df = df.loc[:,df.columns.str.contains("home|date")]
        away_df = df.loc[:,df.columns.str.contains("away|date")]
        team_list = list(set(home_df['home_team']))

        for idx, team in enumerate(team_list):
            a = home_df[home_df['home_team'] == team ].sort_values(by='date').reset_index(drop=True)
            a.columns = a.columns.str.replace("date", "home_date")
            b = away_df[away_df['away_team'] == team ].sort_values(by='date').reset_index(drop=True)
            b.columns = b.columns.str.replace("date", "away_date")
            c = pd.concat((a, b), axis=1)
            
            import os
            directory = "/content/drive/MyDrive/python/Project/EPL_predict/data/team/" + str(team)
            try:
                if not os.path.exists(directory):
                    os.makedirs(directory)
                else:
                    p_c = pd.read_csv(directory + "/{}.csv".format(team), index_col="Unnamed: 0")
                    p_c['date'] = pd.to_datetime(p_c['date'])
                    c = pd.concat([p_c, c], ignore_index=True)
                c.to_csv(directory + "/{}.csv".format(team))
            except OSError:
                print ('Creating directory. ' +  directory)


# 팀별 데이터 rest + outcome
def team_rest_outcome(df, team):
    df_cup = pd.read_csv("/content/drive/MyDrive/python/Project/EPL_predict/data/cup.csv", index_col="Unnamed: 0")
    df_cup['date'] = pd.to_datetime(df_cup['date'])

    cup_team = cup_team(df_cup, team)
    df_team = team_select(df, team) 

    #rest
    team_rest = pd.concat([df_team[cup_team.columns], cup_team], axis=0).sort_values(by='date').reset_index(drop=True)
    rest = (team_rest['date'] - team_rest['date'].shift())
    rest = rest.apply(lambda x: x.days)
    team_rest['rest'] = rest
    team_rest['rest'][0] = rest[rest < 10].median()
    team_clean = pd.merge(df_team, team_rest, on='date', how='left')

    #outcome
    team_clean['outcome'] = (team_clean['home_score'] - team_clean['away_score']).map(outcome)
    return team_clean


def radio_index(df_full):
    df = df_full.copy()
    home_feature = df_full.loc[:, df_full.columns.str.contains("home")]
    home_feature = home_feature.columns.tolist()[1:-4]
    home_feature.remove("home_penalty")
    home_feature.remove("home_throughball")
    away_feature = df_full.loc[:, df_full.columns.str.contains("away")]
    away_feature = away_feature.columns.tolist()[1:-4]
    away_feature.remove("away_penalty")
    away_feature.remove("away_throughball")
    
    for i in range(1, 4):
        df[home_feature[i]] = df_full[home_feature[i]] / (df_full[home_feature[1]] + df_full[home_feature[2]] + df_full[home_feature[3]])
        df[away_feature[i]] = df_full[away_feature[i]] / (df_full[away_feature[1]] + df_full[away_feature[2]] + df_full[away_feature[3]])
        df[home_feature[3+i]] = df_full[home_feature[3+i]] / (df_full[home_feature[4]] + df_full[home_feature[5]] + df_full[home_feature[6]] )
        df[away_feature[3+i]] = df_full[away_feature[3+i]] / (df_full[away_feature[4]] + df_full[away_feature[5]] + df_full[away_feature[6]] )
    
    return df